In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import tensorflow as tf 

2022-12-27 16:50:12.542785: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-27 16:50:12.542809: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
train_dir = "./train/"
test_dir = "./valid/"

In [ ]:
import tensorflow as tf
IMG_SIZE = (224, 224)
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE
                                                                )
                                                                                
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                shuffle=False
                                                               ) 

In [12]:
checkpoint_path = "butterfly"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True, 
                                                         monitor="val_accuracy", 
                                                         save_best_only=True) 

In [15]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential


data_augmentation = Sequential([
  preprocessing.RandomFlip("horizontal"),
  preprocessing.RandomRotation(0.2), 
  preprocessing.RandomHeight(0.2), 
  preprocessing.RandomWidth(0.2), 
  preprocessing.RandomZoom(0.2), 
], name="data_augmentation")

In [16]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False


inputs = layers.Input(shape=(224, 224, 3), name="input_layer") 
x = data_augmentation(inputs)
x = base_model(x, training=False)ract features without updating the weights
x = layers.GlobalAveragePooling2D(name="global_average_pooling")(x) 
outputs = layers.Dense(len(train_data.class_names), activation="softmax", name="output_layer")(x)
model = tf.keras.Model(inputs, outputs)

16719872/16705208 [==============================] - 28s 2us/step


In [17]:
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(), 
              metrics=["accuracy"])

# Fit
history = model.fit(train_data,
                    epochs=5,\
                    validation_data=test_data,
                    validation_steps=int(0.15 * len(test_data)), 
                    callbacks=[checkpoint_callback]
                   )

Epoch 1/5
395/395 [==============================] - 546s 1s/step - loss: 1.7483 - accuracy: 0.6800 - val_loss: 0.8509 - val_accuracy: 0.8281
Epoch 2/5
395/395 [==============================] - 502s 1s/step - loss: 0.6236 - accuracy: 0.8776 - val_loss: 0.5280 - val_accuracy: 0.9219
Epoch 3/5
395/395 [==============================] - 493s 1s/step - loss: 0.4358 - accuracy: 0.9057 - val_loss: 0.4450 - val_accuracy: 0.9375
Epoch 4/5
395/395 [==============================] - 535s 1s/step - loss: 0.3459 - accuracy: 0.9237 - val_loss: 0.4187 - val_accuracy: 0.9375
Epoch 5/5
395/395 [==============================] - 491s 1s/step - loss: 0.2949 - accuracy: 0.9329 - val_loss: 0.3912 - val_accuracy: 0.9531


In [19]:
results_feature_extraction_model = model.evaluate(test_data)
results_feature_extraction_model

16/16 [==============================] - 19s 1s/step - loss: 0.3908 - accuracy: 0.9220


[0.3907508850097656, 0.921999990940094]

In [18]:
model.save('./butterflymodel.h5')